In [1]:
from app.services.audio.audio import AudioSlicer
from pathlib import Path
import io
from app.settings import settings

from app.database_redis.connection import get_redis_client
import pandas as pd
from faster_whisper import WhisperModel
from datetime import datetime, timezone, timedelta
from app.services.audio.redis import Connection, Diarizer, Meeting, Transcriber
from app.settings import settings


path = f"/audio/test.webm"

check_and_process_connections_interval_sec: 1.0


/tmp/ipykernel_2845/551558274.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from app.services.audio.processor import Processor

In [3]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [4]:
current_time = datetime.now(timezone.utc)
meeting_start_timestamp = datetime.now(timezone.utc)-timedelta(seconds=600)

In [5]:
target_slicer = await AudioSlicer.from_ffmpeg_slice(path, 0, 600)
slice_duration = target_slicer.audio.duration_seconds
slice_duration


598.464

In [12]:
segment_start_timestamp = meeting_start_timestamp
segment_end_timestamp = segment_start_timestamp+timedelta(seconds = 10)
user_id = 'me'
meeting_id = 'test2505'
connection_id = 'test'

current_time = datetime.now(timezone.utc)


In [24]:
fileseekstart = (segment_start_timestamp-meeting_start_timestamp).total_seconds()
fileseekend = (segment_end_timestamp-meeting_start_timestamp).total_seconds()

In [25]:
test_slicer = await AudioSlicer.from_ffmpeg_slice(path, fileseekstart, fileseekend)

In [35]:
await test_slicer.export2file('/audio/audio_chunk.webm')

2024-05-25 13:22:26,984 - INFO - app.services.audio.audio - None


In [36]:
test_slicer.audio

In [44]:
connection_id = 'audio_chunk'

In [45]:
connection = Connection(redis_client, connection_id, user_id)
await connection.update_timestamps(segment_start_timestamp, segment_end_timestamp)

In [46]:
#connection.delete_connection_data()

In [47]:
meeting = Meeting(redis_client, meeting_id)

In [48]:
#await meeting.delete_meeting_data()

In [49]:

await meeting.load_from_redis()
await meeting.add_connection(connection.id)
await meeting.set_start_timestamp(segment_start_timestamp)

In [50]:
meeting.transcriber_last_updated_timestamp = (
    meeting.transcriber_last_updated_timestamp or segment_start_timestamp
)

In [51]:
transcriber = Transcriber(redis_client)
await transcriber.add_todo(meeting.meeting_id)
await meeting.update_transcriber_timestamp(
    segment_start_timestamp, transcriber_last_updated_timestamp=current_time
)